In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15
from astropy import units as u
from scipy import stats
from scipy.stats import binned_statistic
import statistics

def bin_data(array_non_0_mass_list, ratio, num_bins): 
    if len(array_non_0_mass_list) == 0:
        return np.array([]), np.array([])

    bin_range = (2e9, 1e13)
    bins = np.geomspace(*bin_range, num_bins + 1)

    bin_means, bin_edges, binnumber = stats.binned_statistic(
        array_non_0_mass_list, ratio, statistic="median", bins=bins
    )
    
    bin_std, _, _ = stats.binned_statistic(
        array_non_0_mass_list, ratio, statistic="std", bins=bins
    )
    
    bin_width = bin_edges[1] - bin_edges[0]
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2 

    valid_indices = bin_means != 0
    binned_x = bin_centers[valid_indices]
    bin_averages = bin_means[valid_indices]
    bin_stds = bin_std[valid_indices]
    
    return bin_stds, binned_x, bin_averages

def snapshot_graph(file_paths, f_catalogs): 
    num_bins = 20
    plt.figure(figsize=(10, 6))

    for file_path, f_catalog in zip(file_paths, f_catalogs):
        mass_list = []
        non_0_mass_list = []
        
        with h5py.File(file_path, "r") as data, h5py.File(f_catalog, 'r') as f:
            redshift = data["Header"].attrs["Redshift"]
            idex_main_subhalo = f['Group/GroupFirstSub'][:]
            halo_masses = f['Group/GroupMass'][:] * 1e10
            subhalo_star_mass = f["Subhalo/SubhaloMassType"][:, 4] * 1e10
            
            for j, mh in enumerate(halo_masses):
                index_of_subhalo = idex_main_subhalo[j]
                mass_of_subhalo = subhalo_star_mass[index_of_subhalo]
           
                if mass_of_subhalo > 0: 
                    mass_list.append(mass_of_subhalo) 
                    non_0_mass_list.append(mh)
                    
        array_non_0_mass_list = np.array(non_0_mass_list)
        ratio = (np.array(mass_list) / array_non_0_mass_list) / 0.16
    
        bin_stds, binned_x, bin_averages = bin_data(array_non_0_mass_list, ratio, num_bins)

        if len(binned_x) < 2 or len(bin_averages) < 2:
            continue

        plt.plot(binned_x, bin_averages, label=f'Simulation: {file_path.split("/")[1]} ')
        plt.scatter(array_non_0_mass_list, ratio,  s = 0.2)
        plt.fill_between(binned_x, bin_averages - bin_stds, bin_averages + bin_stds, alpha=0.2, label=f'Simulation: {file_path.split("/")[1]}')

    plt.xscale("log")
    plt.text(0.7, 0.05, f'Redshift: {redshift}', 
         transform=plt.gca().transAxes, 
         fontsize=12, 
         bbox=dict(facecolor='white', alpha=0.5, edgecolor='black'))
    plt.xlim(1e9, 1e13)
    plt.ylim(0, 0.3)
    plt.xlabel('Halo Masses (Msun)', fontsize=14)
    plt.ylabel('(Star Mass / Halo Mass) / 0.16', fontsize=14)
    plt.legend(fontsize=12)
    
    plt.tight_layout() 
    plt.show()

for i in range(0, 32, 1):
    mass_list = []
    non_0_mass_list = []
        
    if i >= 10:
        result = i
    else:
        result = "0" + str(i)

        # File paths for the three datasets
    file_paths = [
        "data/Output-1M-BH_accsimba_seedcri-SF_/snap_{:03d}.hdf5".format(int(result)),
        "data/Output-1M-BH_accsimba-SF_/snap_{:03d}.hdf5".format(int(result)),
        "data/Output-1M-BH_accsimba_seedcri_seedmass-SF_/snap_{:03d}.hdf5".format(int(result))
    ]

    f_catalogs = [
        "data/Output-1M-BH_accsimba_seedcri-SF_/fof_subhalo_tab_{:03d}.hdf5".format(int(result)),
        "data/Output-1M-BH_accsimba-SF_/fof_subhalo_tab_{:03d}.hdf5".format(int(result)),
        "data/Output-1M-BH_accsimba_seedcri_seedmass-SF_/fof_subhalo_tab_{:03d}.hdf5".format(int(result))
    ]
                
    snapshot_graph(file_paths, f_catalogs)
# Plot SMF of these two and data/Output-1M-BH_accsimba_seedcri_seedmass-SF_
#Make color of fill between and Simulation line the same, limit digites in redshift, check stdiv to make sure it does not collapse